# Lab 2: Flexure of the oceanic lithosphere under island chains 

In this lab, we will solve the plate flexure equation for an ocean island load numerically using finite-differences. We will compare the model results to read data from Hawai'i and estimate the effective elastic thickness of the Pacific plate underneath the islands.

Learning objectives:

* Build on the theorical knowledge acquired in [Lecture 2](https://www.leouieda.com/envs398/slides/2-isostasy/).
* Learn how to apply the finite-difference method to solve partial differential equations.
* Gain a practical understanding of plate flexure.


## General instructions

This is a [Jupyter notebook](https://jupyter.org/) running in [Jupyter Lab](https://jupyterlab.readthedocs.io/en/stable/). The notebook is a programming environment that mixes code (the parts with `[1]: ` or similar next to them) and formatted text/images/equations with [Markdown](https://www.markdownguide.org/basic-syntax) (like this part right here).

Quick start guide:

* **Edit** any cell (blocks of code or text) by double clicking on it.
* **Execute** a code or Markdown cell by typing `Shift + Enter` after selecting it.
* The current active cell is the one with a **blue bar next to it**.
* You can run cells **in any order** as long as the code sequence makes sense (it's best to go top-to-bottom, though).
* To copy any file to the current directory, drag and drop it to the file browser on the left side.
* Notebook files have the extension `.ipynb`.

## Import things

As before, the first thing to do is load the Python libraries that we'll be using. We'll group all our imports here at the top to make it easier to see what we're using.

In [1]:
# The base of the entire scientific Python stack
import numpy as np
# For making plots and figures
import matplotlib.pyplot as plt
# To load and operate on data tables
import pandas as pd

## The plate flexure equation

The equation describing the deflection ($w$) of an elastic plate subject to a vertical load is:

$ D\dfrac{\partial^4 w}{\partial x^4} = q(x) - (\rho_m - \rho_c) g w $

in which $w$ is the deflection (how much the plate bends up or down), $x$ is the position, $D$ is the flexural rigidity of the plate, $rho_c$ and $rho_m$ are the crust and mantle densities, $g$ is gravity (for these purposes, assume $9.8\ m/s^2$, and $q(x)$ is a force density applied by the load on the plate (the weight of the topography).

The flexural rigidity can be described in terms of the plate's elastic parameters:

$D = \dfrac{E T_e^2}{12(1 - \nu^2)}$

$E$ is [Young's modulus](https://en.wikipedia.org/wiki/Young's_modulus), $\nu$ is [Poisson's ratio](https://en.wikipedia.org/wiki/Poisson's_ratio), and $T_e$ is the *effective elastic thickness* (the reference books use $h$ for this but $T_e$ is very common in the literature).

**Note:** For a detailed derivation of these equations and analytical solutions, see "Chapter 3: Elasticity and Flexure" of "Geodynamics" by Turcotte and Schubert.

## The finite-difference method

## Questions

Discuss the following in small groups:

1. Does sea floor spreading happen at the same rate everywhere?
2. Does the maximum earthquake depth indicate the maximum depth of subduction?
3. Which aspect of loading data in Python did you find most difficult?
4. What could be done to make it easier?